In [1]:
# hotel_gui.py
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk
from hotel import Hotel, StandardSuite, DeluxeSuite, PresidentialSuite, Customer

hotel = Hotel("The Grand Heritage")

# Add rooms
for i in range(101, 104):
    hotel.add_room(StandardSuite(i))
for i in range(201, 204):
    hotel.add_room(DeluxeSuite(i))
for i in range(301, 304):
    hotel.add_room(PresidentialSuite(i))

def book_room():
    name = simpledialog.askstring("Name", "Enter full name:")
    email = simpledialog.askstring("Email", "Enter email:")
    phone = simpledialog.askstring("Phone", "Enter 10-digit phone:")
    id_proof = simpledialog.askstring("ID", "Enter ID proof:")

    customer = Customer(name, email, phone, id_proof)
    hotel.add_customer(customer)

    room_type = simpledialog.askstring("Room Type", "1: Standard\n2: Deluxe\n3: Presidential")
    checkin = simpledialog.askstring("Check-in Date", "Enter check-in date (dd/mm/yyyy):")
    duration = simpledialog.askstring("Duration", "Enter stay duration in days:")
    payment = simpledialog.askstring("Payment Mode", "1: UPI\n2: Debit Card\n3: Credit Card\n4: Net Banking\n5: Cash")

    # Simulate existing CLI booking process
    customer_input_func = lambda prompt: {
        "Select Room Type (1/2/3): ": room_type,
        "Enter Check-in Date (dd/mm/yyyy): ": checkin,
        "Enter Stay Duration (in days): ": duration,
        "Enter Payment Mode (1-5): ": payment
    }[prompt]

    # Monkey patch input temporarily (for reuse)
    import builtins
    original_input = builtins.input
    builtins.input = customer_input_func
    hotel.book_room(customer)
    builtins.input = original_input

def cancel_booking():
    reg_id = simpledialog.askinteger("Cancel", "Enter Registration ID:")
    room_no = simpledialog.askinteger("Cancel", "Enter Room No:")

    def cancel_input(prompt):
        return str(reg_id if 'registration' in prompt else room_no)

    import builtins
    original_input = builtins.input
    builtins.input = cancel_input
    hotel.cancel_booking()
    builtins.input = original_input

def show_all_bookings():
    output = ""
    for b in hotel.bookings:
        output += f"Reg No: {b.reg_no}, Name: {b.customer.get_details()['Name']}, Room: {b.room.room_number}\n"
    messagebox.showinfo("Bookings", output or "No bookings yet.")

def print_receipt():
    password = simpledialog.askstring("Admin", "Enter admin password:", show='*')
    if password != "3822":
        messagebox.showerror("Error", "Incorrect password")
        return

    if not hotel.bookings:
        messagebox.showinfo("No Bookings", "No bookings available.")
        return

    reg_id = simpledialog.askinteger("Receipt", "Enter Registration ID:")
    try:
        hotel.generate_pdf_receipt(reg_id)
        messagebox.showinfo("Success", f"Receipt for Reg ID {reg_id} generated.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

def show_admin_bookings():
    password = simpledialog.askstring("Admin", "Enter admin password:", show='*')
    if password != "3822":
        messagebox.showerror("Error", "Incorrect password")
        return

    output = ""
    for b in hotel.bookings:
        total = int(b.duration) * b.room.price
        output += f"Reg No: {b.reg_no}, Name: {b.customer.get_details()['Name']}, Room: {b.room.room_number}, Total: Rs.{total}\n"
    messagebox.showinfo("Admin View", output or "No bookings.")

# GUI setup
root = tk.Tk()
root.title("Hotel Booking System")
root.geometry("400x400")

tk.Label(root, text="The Grand Heritage Hotel", font=("Arial", 16)).pack(pady=10)

tk.Button(root, text="Book Room", width=25, command=book_room).pack(pady=5)
tk.Button(root, text="Cancel Booking", width=25, command=cancel_booking).pack(pady=5)
tk.Button(root, text="Show All Bookings", width=25, command=show_all_bookings).pack(pady=5)
tk.Button(root, text="Show Bookings (Admin)", width=25, command=show_admin_bookings).pack(pady=5)
tk.Button(root, text="Print Receipt (Admin)", width=25, command=print_receipt).pack(pady=5)
tk.Button(root, text="Exit", width=25, command=root.quit).pack(pady=20)

root.mainloop()


ModuleNotFoundError: No module named 'hotel'